## 基于LSTM+CRF的中文命名实体识别

- **任务描述**：命名实体识别是指识别文本中具有特定意义的实体，主要包括人名、地名、机构名、专有名词等。
    - 实现一个简单的命名实体识别方法，该方法通过BiLSTM+CRF模型预测出文本中文字所对应的标签，再根据标签提取出文本中的实体。
    - 从数据文件中加载数据并进行预处理、构建模型、训练模型、评估模型和测试模型。
        - 说明：目前本文档仅作为示例，为了加快训练速度模型较为简单，词向量维度也比较低，因此导致模型准确率较低。

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
import tensorflow_addons as tfa

- tf 2.0 的新特性：eager execution

    - 可以用tf.executing_eargerly()查看Eager Execution当前的启动状态，返回True则是开启，False是关闭。
    - 可以用tf.compat.v1.enable_eager_execution()启动eager模式。
    - 关闭eager模式的函数是 tf.compat.v1.disable_eager_execution()

In [3]:
tf.compat.v1.disable_eager_execution()

### 文本数准备
- 数据说明
- 数据整理
- 加载数据

#### 数据说明
- 数据集共包含约2.7万中文文本，其中包括约2.08万训练集，0.23万验证集和0.46万测试集。
- 数据集分别命名为example.train,example.dev,example.test,保存在datasets目录下。

    - 1.训练集：包含文本和对应的标签，用于模型训练。
    - 2.验证集：包含文本和对应的标签，用于模型训练和参数调试。
    - 3.测试集：包含文本和对应的标签，用于预测结果、验证效果。

- 数据集中标注有三种实体，分别为人名、地名、机构名，标注集合为{'O','B-PER','I-PER','B-ORG','I-ORG','B-LOC','I-LOC'}。
    - 其中'O'表示非实体，
    - 'B-'表示实体的首字，
    - 'I-'表示实体的其他位置的字，
    - 'PER'表示人名，
    - 'ORG'表示机构名，
    - 'LOC'表示地名。IOB（Inside–outside–beginning）是用于标记标志的通用标记格式。
    
#### 数据整理

根据数据集中的字符建立字典，并保存在datasets/vocab.txt中，
标签根据数据集中字符建立字典，并保存在datasets/.txt中

In [4]:
def get_vocab_list(path_list):
    vocab_set = set()
    vocab_list = list()
    for path in path_list:
        with open(path,'r',encoding='utf-8') as f:
            for line in f:
                if len(line.strip()) == 0:
                    continue
                if line[0] not in vocab_set:
                    vocab_set.add(line[0])
                    vocab_list.append(line[0])
    return vocab_list

def save_vocab(path,vocab_list):
    output = ''.join([vocab + '\n' for vocab in vocab_list])
    with open(path,'w', encoding='utf-8') as f:
        f.write(output)
        
vocab_list = get_vocab_list(['./datasets/example.train','./datasets/example.dev','./datasets/example.test'])
save_vocab('./datasets/vocab.txt',vocab_list)

In [5]:
vocab_list

['海',
 '钓',
 '比',
 '赛',
 '地',
 '点',
 '在',
 '厦',
 '门',
 '与',
 '金',
 '之',
 '间',
 '的',
 '域',
 '。',
 '这',
 '座',
 '依',
 '山',
 '傍',
 '水',
 '博',
 '物',
 '馆',
 '由',
 '国',
 '内',
 '一',
 '流',
 '设',
 '计',
 '师',
 '主',
 '持',
 '，',
 '整',
 '个',
 '建',
 '筑',
 '群',
 '精',
 '美',
 '而',
 '恢',
 '宏',
 '但',
 '作',
 '为',
 '共',
 '产',
 '党',
 '员',
 '、',
 '人',
 '民',
 '公',
 '仆',
 '应',
 '当',
 '胸',
 '怀',
 '宽',
 '阔',
 '真',
 '正',
 '做',
 '到',
 '“',
 '先',
 '天',
 '下',
 '忧',
 '后',
 '乐',
 '”',
 '淡',
 '化',
 '名',
 '利',
 '得',
 '失',
 '和',
 '宠',
 '辱',
 '悲',
 '喜',
 '把',
 '改',
 '革',
 '大',
 '业',
 '摆',
 '首',
 '位',
 '样',
 '才',
 '能',
 '超',
 '越',
 '自',
 '我',
 '脱',
 '世',
 '俗',
 '有',
 '所',
 '发',
 '达',
 '家',
 '急',
 '救',
 '保',
 '险',
 '十',
 '分',
 '普',
 '及',
 '已',
 '成',
 '社',
 '会',
 '障',
 '体',
 '系',
 '重',
 '要',
 '组',
 '部',
 '日',
 '俄',
 '两',
 '政',
 '局',
 '都',
 '充',
 '满',
 '变',
 '数',
 '尽',
 '管',
 '关',
 '目',
 '前',
 '是',
 '历',
 '史',
 '最',
 '佳',
 '时',
 '期',
 '其',
 '脆',
 '弱',
 '性',
 '不',
 '言',
 '明',
 '克',
 '马',
 '尔',
 '女',
 '儿',
 '让',
 '娜',
 '今',
 '年'

In [6]:
def get_label_list(path_list):
    label_set = set()
    label_list = list()
    for path in path_list:
        with open(path,'r',encoding='utf-8') as f:
            for line in f:
                if len(line.strip()) == 0:
                    continue
                if line[2:].strip() not in label_set:
                    label_set.add(line[2:].strip())
                    label_list.append(line[2:].strip())
    return label_list

def save_label(path,label_list):
    output = ''.join([label + '\n' for label in label_list])
    with open(path,'w', encoding='utf-8') as f:
        f.write(output)
        
label_list = get_label_list(['./datasets/example.train','./datasets/example.dev','./datasets/example.test'])
save_label('./datasets/label.txt',label_list)

In [7]:
label_list

['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG']

#### 加载数据
分别获取vocab 和label与id的映射

In [8]:
def get_string2id(path):
    string2id = {}
    id2string = []
    with open(path,'r',encoding='utf-8') as f:
        for line in f: 
            string2id[line.strip()] = len(string2id)
            id2string.append(line.strip())
    return id2string,string2id

id2label,label2id = get_string2id("./datasets/label.txt")
id2vocab,vocab2id = get_string2id("./datasets/vocab.txt")

In [9]:
id2label

['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG']

In [10]:
label2id

{'O': 0,
 'B-LOC': 1,
 'I-LOC': 2,
 'B-PER': 3,
 'I-PER': 4,
 'B-ORG': 5,
 'I-ORG': 6}

In [11]:
id2vocab

['海',
 '钓',
 '比',
 '赛',
 '地',
 '点',
 '在',
 '厦',
 '门',
 '与',
 '金',
 '之',
 '间',
 '的',
 '域',
 '。',
 '这',
 '座',
 '依',
 '山',
 '傍',
 '水',
 '博',
 '物',
 '馆',
 '由',
 '国',
 '内',
 '一',
 '流',
 '设',
 '计',
 '师',
 '主',
 '持',
 '，',
 '整',
 '个',
 '建',
 '筑',
 '群',
 '精',
 '美',
 '而',
 '恢',
 '宏',
 '但',
 '作',
 '为',
 '共',
 '产',
 '党',
 '员',
 '、',
 '人',
 '民',
 '公',
 '仆',
 '应',
 '当',
 '胸',
 '怀',
 '宽',
 '阔',
 '真',
 '正',
 '做',
 '到',
 '“',
 '先',
 '天',
 '下',
 '忧',
 '后',
 '乐',
 '”',
 '淡',
 '化',
 '名',
 '利',
 '得',
 '失',
 '和',
 '宠',
 '辱',
 '悲',
 '喜',
 '把',
 '改',
 '革',
 '大',
 '业',
 '摆',
 '首',
 '位',
 '样',
 '才',
 '能',
 '超',
 '越',
 '自',
 '我',
 '脱',
 '世',
 '俗',
 '有',
 '所',
 '发',
 '达',
 '家',
 '急',
 '救',
 '保',
 '险',
 '十',
 '分',
 '普',
 '及',
 '已',
 '成',
 '社',
 '会',
 '障',
 '体',
 '系',
 '重',
 '要',
 '组',
 '部',
 '日',
 '俄',
 '两',
 '政',
 '局',
 '都',
 '充',
 '满',
 '变',
 '数',
 '尽',
 '管',
 '关',
 '目',
 '前',
 '是',
 '历',
 '史',
 '最',
 '佳',
 '时',
 '期',
 '其',
 '脆',
 '弱',
 '性',
 '不',
 '言',
 '明',
 '克',
 '马',
 '尔',
 '女',
 '儿',
 '让',
 '娜',
 '今',
 '年'

In [12]:
vocab2id

{'海': 0,
 '钓': 1,
 '比': 2,
 '赛': 3,
 '地': 4,
 '点': 5,
 '在': 6,
 '厦': 7,
 '门': 8,
 '与': 9,
 '金': 10,
 '之': 11,
 '间': 12,
 '的': 13,
 '域': 14,
 '。': 15,
 '这': 16,
 '座': 17,
 '依': 18,
 '山': 19,
 '傍': 20,
 '水': 21,
 '博': 22,
 '物': 23,
 '馆': 24,
 '由': 25,
 '国': 26,
 '内': 27,
 '一': 28,
 '流': 29,
 '设': 30,
 '计': 31,
 '师': 32,
 '主': 33,
 '持': 34,
 '，': 35,
 '整': 36,
 '个': 37,
 '建': 38,
 '筑': 39,
 '群': 40,
 '精': 41,
 '美': 42,
 '而': 43,
 '恢': 44,
 '宏': 45,
 '但': 46,
 '作': 47,
 '为': 48,
 '共': 49,
 '产': 50,
 '党': 51,
 '员': 52,
 '、': 53,
 '人': 54,
 '民': 55,
 '公': 56,
 '仆': 57,
 '应': 58,
 '当': 59,
 '胸': 60,
 '怀': 61,
 '宽': 62,
 '阔': 63,
 '真': 64,
 '正': 65,
 '做': 66,
 '到': 67,
 '“': 68,
 '先': 69,
 '天': 70,
 '下': 71,
 '忧': 72,
 '后': 73,
 '乐': 74,
 '”': 75,
 '淡': 76,
 '化': 77,
 '名': 78,
 '利': 79,
 '得': 80,
 '失': 81,
 '和': 82,
 '宠': 83,
 '辱': 84,
 '悲': 85,
 '喜': 86,
 '把': 87,
 '改': 88,
 '革': 89,
 '大': 90,
 '业': 91,
 '摆': 92,
 '首': 93,
 '位': 94,
 '样': 95,
 '才': 96,
 '能': 97,
 '超': 98,
 '越': 99,
 '自': 100,

#### 获取每个句子的长度，并得到最大长度

In [13]:
def get_sequence_len(path):
    sequence_len = list()
    tmp_len = 0
    with open(path,'r',encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if len(line) == 0 :
#                 print(tmp_len)
                sequence_len.append(tmp_len)
                tmp_len = 0
            else:
                tmp_len +=1
    return np.array(sequence_len)

In [14]:
train_sequence_len= get_sequence_len('./datasets/example.train')
dev_sequence_len= get_sequence_len('./datasets/example.dev')
test_sequence_len= get_sequence_len('./datasets/example.test')

In [15]:
train_sequence_len,dev_sequence_len,test_sequence_len

(array([18, 35, 88, ..., 53, 65, 41]),
 array([69, 35, 54, ..., 46, 38, 29]),
 array([40, 39, 43, ..., 42, 29, 52]))

In [16]:
max_length = max(max(train_sequence_len),max(dev_sequence_len),max(test_sequence_len))

In [17]:
max_length

577

#### 读取数据，对不足最大长度的句子进行padding

In [18]:
def read_data(path,vocab2id,label2id,max_len):
    data_x = list()
    data_y = list()
    tmp_text = list()
    tmp_label = list()
    
    with open(path,'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if len(line) ==0:
                # 数据集中以空行作为句子分隔符，说明前一个句子已经结束，tmp_text为vocab（id），tmp_label为label(id)
                # padding 至max_len
                tmp_text += [len(vocab2id)] * (max_len - len(tmp_text))
                tmp_label += [0] * (max_len - len(tmp_label))
                
                # 保存已经结束的句子并清空tmp
                data_x.append(tmp_text)
                data_y.append(tmp_label)
                tmp_text = list()
                tmp_label = list()
            else:
                # 将字符和标签转换为相应的id放入tmp
                # 根据数据格式，line[0]为vocab,line[1]为空格
                # line[2:]为label
                tmp_text.append(vocab2id[line[0]])
                tmp_label.append(label2id[line[2:]])
    print('{} include sequences {}'.format(path,len(data_x)))
    return np.array(data_x), np.array(data_y)

In [19]:
train_text,train_label = read_data("./datasets/example.train",vocab2id,label2id,max_length)
dev_text,dev_label = read_data("./datasets/example.dev",vocab2id,label2id,max_length)
test_text,test_label = read_data("./datasets/example.test",vocab2id,label2id,max_length)

./datasets/example.train include sequences 20864
./datasets/example.dev include sequences 2318
./datasets/example.test include sequences 4636


In [20]:
train_text[0],train_label[0]

(array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
           8,   11,   12,   13,    0,   14,   15, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465, 4465,
        4465, 4465, 4465, 4465, 4465, 

#### 讲数据通过tf.data.Dataset加载
Dataset可以看作是相同类型“元素”的有序列表，可以通过Iterator对其中的元素进行读取，通过初始化不同的initializer实现读取不同的数据

In [21]:
train_dataset = tf.compat.v1.data.Dataset.from_tensor_slices((train_text,train_label,train_sequence_len))
train_dataset = train_dataset.batch(1)  #指定读取数据时的batch_size

In [22]:
dev_dataset = tf.compat.v1.data.Dataset.from_tensor_slices((dev_text,dev_label,dev_sequence_len))
dev_dataset = dev_dataset.batch(1)  #指定读取数据时的batch_size

In [23]:
test_dataset = tf.compat.v1.data.Dataset.from_tensor_slices((test_text,test_label,test_sequence_len))
test_dataset = test_dataset.batch(1)  #指定读取数据时的batch_size

In [24]:
iterator = tf.compat.v1.data.Iterator.from_structure(train_dataset.output_types,
                                                     train_dataset.output_shapes)

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


In [25]:
#初始化不同的initializer，从X,Y,S读取的数据不同
train_initializer = iterator.make_initializer(train_dataset)
dev_initializer = iterator.make_initializer(dev_dataset)
test_initializer = iterator.make_initializer(test_dataset)
X,Y,S = iterator.get_next() #运行时每次读取一个batch_size的text,label和sequence_len

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


### 命名实体识别模型构建
- 输入文本经过预处理后得到字符的id，
- 首先经过Embedding层得到字符向量，
- 然后经过BiLSTM层得到句子表示，
- 再经过CRF计算每个字符的标签得分，
- 对于每个字符，选取得分最高的标签作为该字符的类别。

In [26]:
embedding_size = 5
hidden_dim = 4

In [27]:
# embedding层
with tf.compat.v1.variable_scope('embedding'): #随机生成字符向量
    embedding = tf.compat.v1.Variable(
        tf.compat.v1.random_normal([len(vocab2id)+1, embedding_size]),
        dtype= tf.compat.v1.float32
    )
inputs = tf.compat.v1.nn.embedding_lookup(embedding,X) #将X中vocab的id转换为对应的字符向量

In [28]:
# BiLSTM隐藏层
# 前向lstm
lstm_fw_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_dim,reuse = tf.compat.v1.AUTO_REUSE)
# 后向lstm
lstm_bw_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_dim,reuse = tf.compat.v1.AUTO_REUSE)
# 组合成双向lstm
(output,output_states) = tf.compat.v1.nn.bidirectional_dynamic_rnn(
    lstm_fw_cell,
    lstm_bw_cell,
    inputs,
    S,
    dtype=tf.compat.v1.float32
)
#list -> tensor [batch_size,max_len,hidden_dim * 2]
output = tf.stack(output,axis=1)
# -> [batch_size * max_len,hidden_dim * 2]
output = tf.reshape(output,[-1,hidden_dim *2])

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\users\ysilhouette\documents\pyenv\py_365_tf_2\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
c:\users\ysilhouette\documents\pyenv\py_365_tf_2\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [29]:
# BiLSTM输出层
with tf.compat.v1.variable_scope('output'):
    W = tf.compat.v1.Variable(
        tf.compat.v1.truncated_normal(
            shape=[hidden_dim *2 , len(label2id)],
            mean=0,
            stddev=0.1),
        dtype=tf.compat.v1.float32
    )
    B = tf.compat.v1.Variable(
        np.zeros(len(label2id)),
        dtype=tf.compat.v1.float32)
    #[batch_size * max_len,label_size]
    Y_out = tf.matmul(output,W) + B 
scores = tf.reshape(Y_out,[-1, max_length,len(label2id)])  #[batch_size,max_len,label_size]

In [30]:
scores

<tf.Tensor 'Reshape_1:0' shape=(None, 577, 7) dtype=float32>

In [31]:
Y

<tf.Tensor 'IteratorGetNext:1' shape=(None, 577) dtype=int32>

In [32]:
S

<tf.Tensor 'IteratorGetNext:2' shape=(None,) dtype=int32>

In [33]:
# CRF层
with tf.compat.v1.variable_scope('crf',reuse=tf.compat.v1.AUTO_REUSE):
    trans_matrix = tf.compat.v1.get_variable('transition', [len(label2id),len(label2id)])
log_likelihood,_ =tfa.text.crf_log_likelihood(scores,Y,S,trans_matrix)
loss=tf.reduce_mean(-log_likelihood)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.5)
# train_op =optimizer.minimize(loss)#梯度下降法，学习率0.1
train_op = tf.compat.v1.train.GradientDescentOptimizer(0.1).minimize(loss)#梯度下降法，学习率0.1
saver = tf.compat.v1.train.Saver() #用来保存和加载模型
min_Loss = int(1e8) #维护当前最小的Loss,初始化为一个较大的值

### 命名实体识别模型训练
- 通过训练集训练，并保存当前loss值最小的模型。
- 构建的模型抽象出来是一个包含计算和变量的图。定义变量时不会进行运算，也没有具体的值，变量在Session中进行初始化后才有值。
- 在Session中通过run可以执行图，进行相应的计算或取出对应的值。通过训练改变变量的值，最终可以得到模型需要的参数。
- 变量的值仅在一个Session中有效，通过saver保存变量后，可以在新的Session中通过saver加载变量，就无需再重新训练模型。

In [39]:
sess = tf.compat.v1.Session() # 创建一个Session
sess.run(tf.compat.v1.global_variables_initializer()) # 初始化所有变量
for epoch in range(100):
    sess.run(train_initializer) #初始化训练集和initializer
    for step in range(10): #使用了10个batch的数据
        tf_train_scores,tf_trans_matrix,Loss,_ = sess.run(
            [scores,trans_matrix,loss,train_op]
        ) # 执行训练过程
        
    if Loss <min_Loss:
        saver.save(sess,'model/my_model') # 保存当前loss最小的model
        min_Loss = Loss
    print("** epoch",epoch+1,"Loss",Loss)

** epoch 1 Loss 58.72455
** epoch 2 Loss 53.927124
** epoch 3 Loss 50.51535
** epoch 4 Loss 49.02472
** epoch 5 Loss 50.43283
** epoch 6 Loss 50.08667
** epoch 7 Loss 46.792816
** epoch 8 Loss 44.00418
** epoch 9 Loss 42.740723
** epoch 10 Loss 38.633057
** epoch 11 Loss 34.638123
** epoch 12 Loss 30.238342
** epoch 13 Loss 31.892944
** epoch 14 Loss 28.726074
** epoch 15 Loss 27.086182
** epoch 16 Loss 25.640747
** epoch 17 Loss 24.253967
** epoch 18 Loss 23.045715
** epoch 19 Loss 22.389526
** epoch 20 Loss 21.917236
** epoch 21 Loss 21.551575
** epoch 22 Loss 21.257202
** epoch 23 Loss 21.010925
** epoch 24 Loss 20.801147
** epoch 25 Loss 20.622559
** epoch 26 Loss 20.46753
** epoch 27 Loss 20.330994
** epoch 28 Loss 20.209717
** epoch 29 Loss 20.09906
** epoch 30 Loss 20.002686
** epoch 31 Loss 19.913574
** epoch 32 Loss 19.83197
** epoch 33 Loss 19.756409
** epoch 34 Loss 19.690308
** epoch 35 Loss 19.62616
** epoch 36 Loss 19.563904
** epoch 37 Loss 19.511597
** epoch 38 Loss 19.

### 命名实体识别模型评估
#### 计算验证集f1_score并查看验证集预测结果。
- **精确率pression、召回率recall和f1_score**
- 对于结果仅包括“正负”的二分类问题，
    - “预测为正，实际也为正”被称为真阳性，“预测为正，实际为负”被称为假阳性，“预测为负，实际为正”被称为假阴性，
    - 精确率定义为$\frac {真阳性}{真阳性+假阳性}$，召回率定义为$\frac{真阳性}{真阳性+假阴性}$，f1_score可以看作是精确率和召回率的一种加权平均，定义为$ 2*\frac{精确率*召回率}{精确率+召回率}$。
- 对于多分类问题
    - 既可以分别计算出每一类的精确率、召回率和f1_score
        - 计算某一类时可认为该类为“正”，其他所有类为“负”，以此得到每一类的真阳性、假阳性和假阴性数量），再计算出f1_score的平均值，这种f1_score被称为macro-f1。
    - 也可以计算出总的精确率和召回率
        - 得到每一类的真阳性、假阳性和假阴性数量，分别求和得到总的数量），再计算出f1_score，这种f1_score被称为micro-f1
    - 当预测结果未出现错误类别（即未包含在类别集合的类别）时，这种方式计算得到的精确率、召回率和f1_score都相等，值为$\frac{预测正确数量}{预测总数} $。

In [40]:
sess.run(dev_initializer) # 初始化验证集的 initializer
tf_dev_scores,tf_trans_matrix,tf_dev_text,tf_dev_label,tf_dev_sequence_len = sess.run(
    [scores,trans_matrix,X,Y,S]
)
# viterbi_decode每次处理一个句子，这里因为我们设定的batch_size为1，可以直接通过reshape将该维去掉
# 同时去掉句子的padding部分
tf_dev_sequence_len = int(tf_dev_sequence_len)
tf_dev_text = tf_dev_text.reshape(max_length)[:tf_dev_sequence_len]

tf_dev_scores = tf_dev_scores.reshape(max_length,len(label2id))[:tf_dev_sequence_len]
tf_dev_label = tf_dev_label.reshape(max_length)[:tf_dev_sequence_len]

# viterbi_decode传入的scores参数的shape应为[sequence_length,label_size],trans_matix应为[label_size，label_size]
viterbi_sequence,_ = tfa.text.viterbi_decode(tf_dev_scores,tf_trans_matrix)

correct_labels = np.sum(np.equal(viterbi_sequence,tf_dev_label)) # 预测正确的标签数
print("f1_score:{}".format(correct_labels / tf_dev_sequence_len))
print(viterbi_sequence)

f1_score:0.463768115942029
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0]


#### 命名实体识别模型预测
##### 加载模型

In [43]:
sess = tf.compat.v1.Session()
saver.restore(sess,"model/my_model") #加载模型参数
sess.run(test_initializer)  # 初始化测试集的initializer

tf_test_scores,tf_test_matrix,tf_test_text,tf_test_label,tf_test_sequence_len = sess.run(
    [scores,trans_matrix,X,Y,S]
)
tf_test_sequence_len = int(tf_test_sequence_len)
tf_test_text = tf_test_text.reshape(max_length)[:tf_test_sequence_len]

tf_test_scores = tf_test_scores.reshape(max_length,len(label2id))[:tf_test_sequence_len]
tf_test_label = tf_test_label.reshape(max_length)[:tf_test_sequence_len]

viterbi_sequence,_ = tfa.text.viterbi_decode(tf_test_scores,tf_trans_matrix)

example_test_text = [id2vocab[vocab] for vocab in tf_test_text] # id转换为vocab
example_prediction_label = [id2label[label] for label in viterbi_sequence] # id转换为label

print([id2vocab[tf_test_text[index]] + '' + id2label[viterbi_sequence[index]] for index in range(len(tf_test_text))])
sess.close()

INFO:tensorflow:Restoring parameters from model/my_model
['我I-ORG', '们I-ORG', '变I-ORG', '而O', '以O', '书O', '会O', '友O', '，O', '以O', '书O', '结O', '缘O', '，O', '把O', '欧O', '美O', '、O', '港O', '台O', '流B-ORG', '行I-ORG', '的I-ORG', '食I-ORG', '品I-ORG', '类I-ORG', '图I-ORG', '谱I-ORG', '、I-ORG', '画I-ORG', '册I-ORG', '、I-ORG', '工I-ORG', '具I-ORG', '书I-ORG', '汇I-ORG', '集I-ORG', '一I-ORG', '堂I-ORG', '。O']


In [44]:
example_test_text

['我',
 '们',
 '变',
 '而',
 '以',
 '书',
 '会',
 '友',
 '，',
 '以',
 '书',
 '结',
 '缘',
 '，',
 '把',
 '欧',
 '美',
 '、',
 '港',
 '台',
 '流',
 '行',
 '的',
 '食',
 '品',
 '类',
 '图',
 '谱',
 '、',
 '画',
 '册',
 '、',
 '工',
 '具',
 '书',
 '汇',
 '集',
 '一',
 '堂',
 '。']

In [45]:
example_prediction_label 

['I-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O']

#### 结果展示

In [46]:
exam_test_text = ['我','们','变','而','以','书','会','友','，','以','书','结','缘','，',
                     '把','欧','美','、','港','台','流','行','的','食','品','类','图','谱','、',
                     '画','册','、','工','具','书','汇','集','一','堂','。']
exam_prediction_label = ['O','O','O','O','O','O','O','O','O','O','O','O','O','O',
                            'O','B-LOC','B-LOC','O','B-LOC','B-LOC','O','O','O','O','O','O','O','O','O',
                            'O','O','O','O','O','O','O','O','O','O','O']

In [52]:
def get_named_entity(text,labels):
    named_entity_set = set() #去重，同一个实体只记录一次
    named_entity_list = list()
    cur_type= ''
    is_entity = False
    tmp_named_entity= ''
    
    for index in range(len(text)):
        label = labels[index]
        if label =='O' or label[:2] == 'B-': # 可能是前一个实体的结束
            if is_entity == True: # 一个实体的结束
                if tmp_named_entity not in named_entity_set:
                    named_entity_set.add(tmp_named_entity)
                    named_entity_list.append({'text':tmp_named_entity,'type':cur_type})
                
                is_entity =False
                tmp_named_entity =''
            if label == 'O':
                continue
            cur_type = label[2:] # 若为'B-',说明此时是回一个实体的开头
            is_entity =True
            tmp_named_entity += text[index]
        else: # 'I-'
            tmp_named_entity += text[index]
    return named_entity_list

In [53]:
named_entity_list = get_named_entity(exam_test_text,exam_prediction_label)
print(named_entity_list)

[{'text': '欧', 'type': 'LOC'}, {'text': '美', 'type': 'LOC'}, {'text': '港', 'type': 'LOC'}, {'text': '台', 'type': 'LOC'}]
